In [19]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [20]:
train = pd.read_csv('DATA/train.csv', header=0, encoding = "ISO-8859-1")

In [21]:
# Since relevance is an aggregate, values are continuous rather than discrete
# It appears the classifier can only handle a limited number of values
# So we round up
train['relevance_rounded'] = train['relevance'].map(lambda x: round(x))

In [22]:
# Create the feature 'exact match', allowing only for differences in case
train['exact'] = pd.Series([True if st.lower() in pt.lower() else False 
                      for st, pt in zip(train['search_term'], train['product_title'])])

In [27]:
# Prepare to divide the dataset into test, train (lifted from sklearn Iris example)
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75

# Split data into training and test
train_data, test_data = train[train['is_train']==True], train[train['is_train']==False]

#features = df.columns[['exact']] # Whether it's an exact match or not

features = ['exact']

clf = RandomForestClassifier(n_jobs=2)
y, _ = pd.factorize(train_data['relevance_rounded']) # 0, 1, 2

clf.fit(train_data[features], y)

# Predicting on those features will output predictions that match y
preds = clf.predict(test_data[features])

target_names = test_data['relevance_rounded']
preds = target_names[clf.predict(test_data[features])]
preds.index = range(len(preds))
pd.crosstab(test_data['relevance_rounded'], preds, rownames=['actual'], colnames=['preds'])

preds,2.0
actual,
1,260
2,1770
3,2114
